In [ ]:
# Ontology Designer


import types
from owlready2 import get_ontology, Thing, ObjectPropertyClass
import rdflib
import os
import io
import warnings

warnings.filterwarnings("ignore")

# Temporary Ontology Link
# ONTOLOGY_PATH = r'https://github.com/gryvity/MIAPPEx/onto_build/lab_ontologies/miappeXsosa.owl'
ONTOLOGY_PATH = r'miappeXsosa/miappeXsosa.owl'

# Import existiing Ontologies
IMPORT_LOCAL = {
    'ppeo' : os.path.join(r'external/ppeo.owl'),
    'sosa' : os.path.join(r'external/sosa.owl'), 
    'ssn'  : os.path.join(r'external/ssn.ttl')
    }


# THESE NEED A REDESIGN AS WELL
ONTOLOGIES_IMPORT = {
    'ppeo' : [],
    'sosa' : [],
    'ssn'  : []
}

CLASSES = ONTOLOGIES_IMPORT.copy()
# DATA_PROPERTIES = ONTOLOGIES_IMPORT.copy()
# OBJECT_PROPERTIES = ONTOLOGIES_IMPORT.copy()

# PROPERTIES = {'domain': None, 'range': None, 'name': None}

# set classes


# FOR NOW WE WILL PICK A FEW CLASSES TO TEST THE FUNCTIONALITY (LATER WE WILL IMPORT EVERYTHING)
CLASSES['ppeo'].extend(
    [
    'investigation',
    'person',
    'study',
    'observation_unit',
    'observed_variable',
    'biological_material',
    'factor',
    'environment'
    ]
)

CLASSES['sosa'].extend(
    [
    'Observation',
    'FeatureOfInterest',
    'Procedure',
    'ObservableProperty',
    'Sensor',
    'Platform'
    ]
)

CLASSES['ssn'].extend(
    [
    'System',
    'Input',
    'Output',
    'Deployment'
    ]
)




class OntologyBuilder(object):
    IMPORT = {
        'ppeo' : ('http://ppeo.org/ontology/ppeo.owl', 'owl'),
        'sosa' : ('http://www.w3.org/ns/sosa/', 'ttl'),
        'ssn'  : ('http://www.w3.org/ns/ssn/', 'ttl')
    }

    def __init__(self, onto_IRI=ONTOLOGY_PATH, onto_path=ONTOLOGY_PATH, import_local=IMPORT_LOCAL, refresh=True):

        self.onto_path = onto_path

        """Here let us collect all available classes and object properties in form as 
        
        [classname] : {
            'superclasses' : [list of superclasses (is_a)],
            'iri' : class_iri,
            'name' : classname,
            'comment' : comment,
            'index' : index in imported ontologies,
            'imported_from' : self.onto.imported_ontologies[index].name

            'object_properties' : {
                ['property_name'] : {
                    'iri' : property_iri,
                    'comment' : comment,
                    'domain' : domain,
                    'range' : range}
                }
            'data_properties' : {
                ['property_name'] : {
                    'iri' : property_iri,
                    'comment' : comment,
                    'domain' : domain,
                    'range' : range}
                }
        
        """
        self.onto_structure = {

        }

        if os.path.exists(onto_path) and not refresh:
            self.onto = get_ontology(f'file://{os.path.abspath(onto_path)}').load()
        else:
            self.onto = get_ontology(onto_IRI)

        if refresh:
            self.onto.imported_ontologies.clear()
        
       
        if import_local:
            for label, path in import_local.items():
                self.onto.imported_ontologies.append(self._load_local_import(path))
                self.onto.imported_ontologies[-1].name = label

        self._load_import_content()

    def _load_import_content(self):
        pass
       
    def _load_local_import(self, path):
        match path.split('.')[-1].lower():
            case 'owl':
                return get_ontology(path).load()
            case 'ttl':
                g = rdflib.Graph()
                g.parse(path, format='turtle')
                with io.BytesIO(g.serialize(format='pretty-xml').encode('utf-8')) as temp_file:
                    return get_ontology(path).load(fileobj=temp_file)
                
            case _:
                raise ValueError(f'Unsupported file format: {path}')
    
    def save(self, path=None):
        if path is None:
            path = self.onto_path
        else:
            if not os.path.exists(os.path.dirname(path)):
                os.makedirs(os.path.dirname(path))
        self.onto.save(file = path, format = 'rdfxml')

    def unify_name(self, name):
        if name.islower():
            name = name.title()
        if '_' in name:
            return name.replace('_', ' ').title().replace(' ', '')
        else:
            return name

    def _load_classes_test(self):
        for ont in self.onto.imported_ontologies:
            if ont.name in CLASSES:
                for class_name in ont.classes():
                    if class_name.name in CLASSES[ont.name]:
                        self.make_class(self.unify_name(class_name.name), parent_class=class_name)

    def make_class(self, class_name, parent_class=Thing):
        with self.onto:
            NewClass = types.new_class(class_name, (parent_class,))
            NewClass.is_a = [parent_class]
         
        return NewClass
    


        


# Test

onto = OntologyBuilder()

onto.save()
    

print([test.name for test in onto.onto.get_imported_ontologies()])

ns = 'ppeo'
cn = 'investigation'

# Step 1: Identify Index of imported Ontology
inx = [o.name for o in onto.onto.get_imported_ontologies()].index(ns)

picked_cl = [c for c in onto.onto.get_imported_ontologies()[inx].classes() if c.name == cn].pop()

print(picked_cl.name)
for prop in [p for p in picked_cl.get_class_properties() if isinstance(p, ObjectPropertyClass)]:
    print(f' - {prop.name}, domain: {prop.domain}, range: {prop.range}, type: {type(prop)}')

# Based on the code above we redesign the following mehtods

# class Test(OntologyBuilder):
#     def __init__(self, onto_IRI=ONTOLOGY_PATH, onto_path=ONTOLOGY_PATH, import_local=IMPORT_LOCAL, refresh=True):
#         super().__init__(onto_IRI, onto_path, import_local, refresh )
#         self.onto_content = {}

#     def _load_ontology_content(self):
#         for ont 


"""
    def _load_classes(self):
        for ont in self.onto.imported_ontologies:
            if ont.name in CLASSES:
                for class_name in ont.classes():
                    if class_name.name in CLASSES[ont.name]:
                        self.make_class(self.unify_name(class_name.name), parent_class=class_name)

    def make_class(self, class_name, parent_class=Thing):
        with self.onto:
            NewClass = types.new_class(class_name, (parent_class,))
            NewClass.is_a = [parent_class]
         
        return NewClass
 """


['ppeo', 'sosa', 'ssn']
investigation
 - hasPersonWithRole, domain: [ppeo.investigation | ppeo.study], range: [ppeo.role], type: <class 'owlready2.prop.ObjectPropertyClass'>
 - hasPart, domain: [], range: [], type: <class 'owlready2.prop.ObjectPropertyClass'>


'\n    def _load_classes(self):\n        for ont in self.onto.imported_ontologies:\n            if ont.name in CLASSES:\n                for class_name in ont.classes():\n                    if class_name.name in CLASSES[ont.name]:\n                        self.make_class(self.unify_name(class_name.name), parent_class=class_name)\n\n    def make_class(self, class_name, parent_class=Thing):\n        with self.onto:\n            NewClass = types.new_class(class_name, (parent_class,))\n            NewClass.is_a = [parent_class]\n\n        return NewClass\n '

# Strategy

- Make a new ontology which imports a small sett of classes from sosa, ssn and ppeo
- Find a visualization tool (or draw a data model yourself)
- For each class create a json-object which serves as a container of data
- Write a converter from MIAPPE tables to JSON-LD format


In [31]:
# Try to load an ontology from the internet
from owlready2 import get_ontology
import rdflib

test_onto1 = get_ontology('http://purl.org/ppeo/PPEO.owl').load()


g = rdflib.Graph()
g.parse('https://www.w3.org/ns/ssn/', format='turtle')

with io.BytesIO(g.serialize(format='pretty-xml').encode('utf-8')) as temp_file:
    test_onto2 = get_ontology('http://www.w3.org/ns/ssn/').load(fileobj=temp_file)

g = rdflib.Graph()
g.parse('https://www.w3.org/ns/sosa/', format='turtle')
with io.BytesIO(g.serialize(format='pretty-xml').encode('utf-8')) as temp_file:
    test_onto3 = get_ontology('http://www.w3.org/ns/sosa/').load(fileobj=temp_file)

for cls in test_onto3.classes():
    if str(cls).startswith('sosa'):
        print(cls)

sosa.ActuatableProperty
sosa.Actuation
sosa.Actuator
sosa.FeatureOfInterest
sosa.ObservableProperty
sosa.Observation
sosa.Platform
sosa.Procedure
sosa.Result
sosa.Sample
sosa.Sampler
sosa.Sampling
sosa.Sensor
